In [ ]:
######################
# Initializing codes #
######################

print('프로그램 준비 중 입니다. 다른 코드를 실행하지 마세요.')

# Import packages
from io import BytesIO
from picamera import PiCamera
from time import sleep
from PIL import Image, ImageOps
import tensorflow.keras
import numpy as np

# Make label dict
label = {}
with open('labels.txt', 'r') as label_file:
    for line in label_file:
        symbol, name = line.split(' ')
        label[symbol] = name.rstrip()
print(label)

# Init Pi camera
camera = PiCamera()
camera.rotation = 180

# Load keras model
np.set_printoptions(suppress=True)
model = tensorflow.keras.models.load_model('keras_model.h5')
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
size = (224, 224)

print('준비가 끝났습니다. 사진 파일 불러오기나 카메라로 사진찍기 중 하나를 실행하세요.')

In [ ]:
# 사진 파일 불러오기
# Read image file
image = Image.open('test_photo.jpg')

In [ ]:
# 카메라로 사진 찍기
# Capture from Pi camera
camera.start_preview()
stream = BytesIO()
sleep(0.25)
camera.capture(stream, 'jpeg')
stream.seek(0)
image = Image.open(stream)

In [ ]:
# 불러온 사진 판별하기
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)

In [ ]:
# 결과를 출력하기
# print readable prediction
def print_prediction(label, prediction):
    for index, prob in enumerate(prediction[0]):
        print(label[str(index)], '%.2f' % (prob*100) + '%')
        
# return highest predicted label name
def get_predicted_label(label, prediction):
    probs = list(prediction[0])
    highest_index = probs.index(max(probs))
    return label[str(highest_index)]
        
print_prediction(label, prediction)
print(get_predicted_label(label, prediction))